In [1]:
from crewai import Agent, Task, Crew
from langchain_community.tools import DuckDuckGoSearchRun

In [27]:
from langchain_community.llms import Ollama
# ollama_llm = Ollama(model="openhermes") # assuming you have Ollama installed and have openhermes model pulled with `ollama pull openhermes`
# ollama_llm = Ollama(model="phi")
ollama_llm = Ollama(model="llama3")

In [36]:
from crewai_tools import tool
@tool("Duck_Duck_Go_Search")
def ddgsearch(question: str) -> str:
    """Clear description for what this tool is useful for, your agent will need this information to use it."""
    # Function logic here
    return DuckDuckGoSearchRun().run(question)

In [37]:
# What is it that needs to get done?
research_topic = "Differences between open source large language models and closed source large language models."

In [38]:
researcher = Agent(
  role='Researcher',
  goal='Search the internet for information about {research_topic}',
  backstory="""
  You are a researcher. Using the information in the task, you find out some of the most popular facts about the topic along with some of the trending aspects.
  You provide a lot of information thereby allowing a choice in the content selected for the final blog.
  """,
  verbose=True,
  allow_delegation=False,
  # tools=[DuckDuckGoSearchRun()],
  tools=[ddgsearch],
  llm=ollama_llm
)

In [39]:
writer = Agent(
  role='Tech Content Strategist',
  goal='Craft compelling content on a set of information provided by the researcher.',
  backstory="""You are a writer known for your humorous but informative way of explaining. 
  You transform complex concepts into compelling narratives.""",
  verbose=True,
  allow_delegation=True,
  llm=ollama_llm
)

In [40]:
task1 = Task(
  agent=researcher,
  description=research_topic,
  expected_output="A complete analysis of the {research_topic}, presented as a report.",
)

In [41]:
task2 = Task(
  agent=writer,
  description="""Using the insights provided, develop an engaging blog
  post that highlights the most significant facts and differences between open-source LLMs and closed-source LLMs.
  Your post should be informative yet accessible, catering to a tech-savvy audience.
  Make it sound simple, and avoid complex words so it doesn't sound AI generated.""",
  expected_output="""A article containing a minimum of 4 paragraphs. contains no more than 2500 words but no less than 1500 words.
  The final output will not contain any extra fluff like "Paragraph 1:" or any action the writer should do.""",
)

In [42]:
# Instantiate your crew with a sequential process
crew = Crew(
  agents=[researcher, writer],
  tasks=[task1, task2],
  verbose=2, # You can set it to 1 or 2 for different logging levels
)

2024-06-24 09:01:50,914 - 139688074141696 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [43]:
# Get your crew to work!
result = crew.kickoff()

 [2024-06-24 09:01:51][DEBUG]: == Working Agent: Researcher
 [2024-06-24 09:01:51][INFO]: == Starting Task: Differences between open source large language models and closed source large language models.


> Entering new CrewAgentExecutor chain...
Thought: It's time to start searching for information about the differences between open source large language models and closed source large language models.

Action: Duck_Duck_Go_Search

Action Input: {"q": "what are the key differences between open source and closed source large language models 

I encountered an error while trying to use the tool. This was the error: ddgsearch() got an unexpected keyword argument 'q'.
 Tool Duck_Duck_Go_Search accepts these inputs: Duck_Duck_Go_Search(question: 'string') - Clear description for what this tool is useful for, your agent will need this information to use it.

I will proceed with caution, taking into account the error message from the Duck_Duck_Go_Search tool.

Thought: I need to reformulate m

In [17]:
result

"Open-source and closed-source large language models (LLMs) have distinct differences in terms of accessibility, customization, licensing restrictions, cost, innovation, scalability, and vendor lock-in. Open-source LLMs provide greater freedom, flexibility, and control over code modifications without the constraints of commercial licenses. They are accessible to anyone who wishes to inspect or modify the code, but may require in-house expertise for maintenance and updates.\n\nOpen-source LLMs offer a diverse range of models and approaches due to community contributions, which can result in slower innovation compared to closed-source LLMs developed by large corporations with extensive research and development investments. These corporate solutions often deliver innovative capabilities and industry-leading performance but come with significant licensing fees and potential vendor lock-in concerns.\n\nChoosing between open-source and closed-source LLMs depends on your organization's specif

In [12]:
with open("Output.txt", "w") as text_file:
    print(result, file=text_file)